# ***This is a simple implement of basic 1st attack introduced in ML-leak article on MNIST***

## ***General setting code***

### ***Install all independency here***

In [1]:
!pip3 install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
     |████████████████████████████████| 199.3 MB 2.1 MB/s 
     |████████████████████████████████| 16.1 MB 85.6 MB/s 
     |████████████████████████████████| 2.7 MB 8.9 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires 

### ***Import all dependency here***

In [1]:
# to monitor the progress
from tqdm import tqdm
import time
# basic dependency
import numpy as np
import random
# pytorch related
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import optim
import torch.nn.functional as F
import torchvision.transforms as T
# for visulization
import matplotlib.pyplot as plt
from random import shuffle
# sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# other classifier
import lightgbm as lgb
# for mount from Colab to my drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Mounted at /content/drive


## ***Target and shadow model code***

### ***Define a Neural Network class for shadow and target model***

In [2]:
# reuse the NN in badnet
class basicNet(nn.Module):

    def __init__(self,inputchannels,outputclasses):
        super().__init__()
        self.conv1 = nn.Conv2d(inputchannels, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool = nn.AvgPool2d(2)
        if inputchannels == 3:
          inputfeatures = 800
        else:
          inputfeatures = 512
        self.fc1 = nn.Linear(inputfeatures, 512)
        self.fc2 = nn.Linear(512, outputclasses)

    def forward(self, x):
        # conv block1
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        # conv block2
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        # reshape(flat) the feature to be the input of full connect layer
        x = x.view(-1, self.num_features(x))
        # fc block1
        x = self.fc1(x)
        x = F.relu(x)
        # fc block2
        x = self.fc2(x)
        x = F.softmax(x,dim=-1)
        return x

    def num_features(self, x):
        # size of different dimensions
        size_D = x.size()[1:]
        total = 1
        for i in size_D:
            total = total*i
        return total

### ***Functions for training and evaluating(target/shadow model)***

In [3]:
def train(model, dataloader, criterion, opt):
    running_loss = 0
    # switch to model:train
    # no difference here since no dropout and BN
    model.train()
    count = 0
    for i, data in tqdm(enumerate(dataloader)):
        opt.zero_grad()
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        predict = model(imgs)
        #print(predict.dtype)
        #print(labels.dtype)
        loss = criterion(predict, labels)
        loss.backward()
        opt.step()
        count = i
        running_loss += loss
    return running_loss / count


def evaluation(model, dataloader, batch_size=64):
    # switch to model:eval
    # no difference here since no dropout and BN    
    model.eval()
    # y_tensorlist is a list consists of some tensors
    y_true_tensorlist = []
    y_predict_tensorlist = []
    for step, (batch_x, batch_y) in enumerate(dataloader):
        batch_x,batch_y = batch_x.to(device),batch_y.to(device)
        output = model(batch_x)
        #print(output.shape)
        batch_y_predict = torch.argmax(output, dim=1)
        
        y_predict_tensorlist.append(batch_y_predict)
        
        y_true_tensorlist.append(batch_y)
        
    # combine the tensors in the list into one
    y_true = torch.cat(y_true_tensorlist,0)
    y_predict = torch.cat(y_predict_tensorlist,0)

    # compute accuracy
    length = len(y_true)
    right_length = torch.sum(y_true == y_predict)
    #print(right_length/length)
    
    return right_length/length

## ***Attack model's dataset code***

### ***Functions used to create attack model's dataset***

In [4]:
def attackdataset_x_1_batch(shadow,batch_x,device):
    shadow.eval()
    # Aim to get features(x)
    # output of model shadow is probabilities
    batch_y = shadow(batch_x.to(device))
    sorted_prob,_ = torch.sort(batch_y,descending=True)
    #print(sorted_prob.shape)

    # choose 3 highest probabilities as feature vector
    # This is a batch's output, not single image.
    # feature: x, x is already a tensor
    feature_vectors = torch.narrow(sorted_prob,dim=1,start=0,length=3)

    return feature_vectors

def get_attackdataset(shadow,train_loader,out_loader,device):
    shadow.eval()
    # 1. get a tensor list of x
    # 2. combine all tensors in tensor list x into a tensor X
    # 3. generate label tensor Y, the length is the same as X
    attack_set=[]

    attack_X_train_list = []
    for i, (batch_x, batch_y) in enumerate(train_loader):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        feature_vectors = attackdataset_x_1_batch(shadow,batch_x,device=device)
        attack_X_train_list.append(feature_vectors)
    attack_X_train = torch.cat(attack_X_train_list,0)
    for i in attack_X_train:
      attack_set.append((i.detach(),torch.ones(1)))

    # train:label = 1
    #attack_Y_train = torch.ones(attack_X_train.shape[0])
    #print('attack Y train shape: ',attack_Y_train.shape)

    attack_X_out_list = []
    for i, (batch_x, batch_y) in enumerate(out_loader):
        feature_vectors = attackdataset_x_1_batch(shadow,batch_x,device=device)
        attack_X_out_list.append(feature_vectors)
    attack_X_out = torch.cat(attack_X_out_list,0)
    for i in attack_X_out:
      attack_set.append((i.detach(),torch.zeros(1)))
    # out: label = 0
    #attack_Y_out = torch.zeros(attack_X_out.shape[0])
    #print('attack Y out shape: ',attack_Y_out.shape)

    # combine train and out to get the whole dataset's X and Y
    #attack_X = torch.cat([attack_X_train,attack_X_out],0)
    #attack_Y = torch.cat([attack_Y_train,attack_Y_out],0)
    #print('attack X total shape: ',attack_X.shape)
    #return attack_X.detach(),attack_Y
    shuffle(attack_set)
    print(attack_set[0])
    return attack_set

### ***Define attack model's dataset class***

In [5]:
class attack_dataset(Dataset):
    # N data points, features include N top 3 probabilities, labels include N labels(1/0)
    # elements in features and labels are already tensor.
    def __init__(self, attack_set, transform = None, device=torch.device("cpu")):
        self.attack_set = attack_set
        self.device = device
        self.transform = transform

    def __getitem__(self, item):
        # extract x,y , x,y are all tensors
        x = self.attack_set[item][0]
        y = self.attack_set[item][1]
        # send x,y to device
        x = x.to(self.device)
        y = y.to(self.device)
        
        return x, y

    def __len__(self):
        return len(self.attack_set)

## ***Main part***

### ***MNIST***

#### *Download, split and load data*

In [9]:
# prepare original data
transforms = T.Compose([T.ToTensor(),T.Normalize((0.1307,), (0.3081,))])
train_data_MNIST = datasets.MNIST(root="./data_1/", train=True,transform=transforms, download=True)
test_data_MNIST = datasets.MNIST(root="./data_1/",train=False,transform=transforms, download=True)

In [10]:
# set the length of each split
length_train_data_MNIST = len(train_data_MNIST)
lengthlist = [int(0.25*length_train_data_MNIST),int(0.25*length_train_data_MNIST),int(0.25*length_train_data_MNIST),int(0.25*length_train_data_MNIST)]
print(length_train_data_MNIST)
print(lengthlist)

60000
[15000, 15000, 15000, 15000]


In [11]:
# MNIST
# split the dataset into 4 part: target train/out, shadow train/out
train_target_dataset_MNIST,out_target_dataset_MNIST,train_shadow_dataset_MNIST,out_shadow_dataset_MNIST = random_split(
    dataset=train_data_MNIST,
    lengths=lengthlist,
    generator=torch.Generator().manual_seed(19260817)
)
# This is the main dataloader with the total dataset
target_train_loader = DataLoader(dataset=train_target_dataset_MNIST, batch_size=64, shuffle = True)
target_out_loader = DataLoader(dataset=out_target_dataset_MNIST, batch_size=64, shuffle = True)

shadow_train_loader = DataLoader(dataset=train_shadow_dataset_MNIST, batch_size=64, shuffle = True)
shadow_out_loader = DataLoader(dataset=out_shadow_dataset_MNIST, batch_size=64, shuffle = True)

testloader = DataLoader(dataset=test_data_MNIST, batch_size=64, shuffle = True)


#### *Pretrain target and shadow models*

In [10]:
# use train_target to train target model
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Train target
# load model
# if use MNIST, inputchannels=1, if use CIFAR10, inputchannels=3, both output classes = 10
# related info could be get by using input_channels=train_data_loader.dataset.channels, output_num=train_data_loader.dataset.class_num
target = basicNet(inputchannels=1, outputclasses=10).to(device)
# settings
criterion = nn.CrossEntropyLoss()
sgd = optim.SGD(target.parameters(), lr=0.01, momentum=0.9)
epoch = 20

# train
print("start training: ")
for i in range(epoch):
    loss_train = train(target, target_train_loader, criterion, sgd)
# Acc
acc_train = evaluation(target, target_train_loader, batch_size=64)
acc_test = evaluation(target, testloader, batch_size=64)
print("epoch%d   loss: %.5f  training accuracy: %.5f  testing accuracy: %.5f"\
      % (epoch, loss_train, acc_train, acc_test))
torch.save(target.state_dict(), "./models/target_MNIST.pth")

start training: 


235it [00:07, 30.75it/s]
235it [00:07, 31.36it/s]
235it [00:07, 30.80it/s]
235it [00:08, 26.34it/s]
235it [00:07, 30.70it/s]
235it [00:07, 30.86it/s]
235it [00:07, 30.34it/s]
235it [00:07, 30.99it/s]
235it [00:08, 28.55it/s]
235it [00:07, 31.13it/s]
235it [00:07, 31.32it/s]
235it [00:07, 31.10it/s]
235it [00:07, 30.96it/s]
235it [00:07, 31.42it/s]
235it [00:07, 31.36it/s]
235it [00:07, 31.34it/s]
235it [00:07, 31.11it/s]
235it [00:07, 30.83it/s]
235it [00:07, 31.14it/s]
235it [00:07, 30.96it/s]


epoch20   loss: 1.48706  training accuracy: 0.98247  testing accuracy: 0.97420


In [12]:
# use train_shadow to train shadow model
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Train shadow
# load model
# if use MNIST, inputchannels=1, if use CIFAR10, inputchannels=3, both output classes = 10
# related info could be get by using input_channels=train_data_loader.dataset.channels, output_num=train_data_loader.dataset.class_num
shadow = basicNet(inputchannels=1, outputclasses=10).to(device)
# settings
criterion = nn.CrossEntropyLoss()
sgd = optim.SGD(shadow.parameters(), lr=0.01, momentum=0.9)
epoch = 20

# train
print("start training: ")
for i in range(epoch):
    loss_train = train(shadow, shadow_train_loader, criterion, sgd)
# Acc
acc_train = evaluation(shadow, shadow_train_loader, batch_size=64)
acc_test = evaluation(shadow, testloader, batch_size=64)
print("epoch%d   loss: %.5f  training accuracy: %.5f  testing accuracy: %.5f"\
      % (epoch, loss_train, acc_train, acc_test))
torch.save(shadow.state_dict(), "./models/shadow_MNIST.pth")

start training: 


235it [00:07, 31.46it/s]
235it [00:07, 31.31it/s]
235it [00:07, 31.30it/s]
235it [00:07, 31.29it/s]
235it [00:07, 31.12it/s]
235it [00:07, 31.53it/s]
235it [00:07, 31.41it/s]
235it [00:07, 31.28it/s]
235it [00:08, 27.58it/s]
235it [00:07, 29.85it/s]
235it [00:13, 16.90it/s]
235it [00:09, 23.55it/s]
235it [00:08, 26.92it/s]
235it [00:07, 29.90it/s]
235it [00:09, 24.84it/s]
235it [00:09, 24.92it/s]
235it [00:07, 30.02it/s]
235it [00:07, 30.60it/s]
235it [00:09, 24.48it/s]
235it [00:07, 29.39it/s]


epoch20   loss: 1.48643  training accuracy: 0.98620  testing accuracy: 0.97560


#### *Load pretrained target and shadow models (to save time)*

In [12]:
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# load pretrained model
target_pre = basicNet(inputchannels=1, outputclasses=10).to(device)
# if gpu, remove map_location=torch.device('cpu')
target_pre.load_state_dict(torch.load("./models/target_MNIST.pth",map_location=torch.device('cpu')))
target_pre.eval()
shadow_pre = basicNet(inputchannels=1, outputclasses=10).to(device)
# if gpu, remove map_location=torch.device('cpu')
shadow_pre.load_state_dict(torch.load("./models/shadow_MNIST.pth",map_location=torch.device('cpu')))
shadow_pre.eval()

basicNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [13]:
# test if these models are loaded
acc_test1 = evaluation(target_pre, testloader, batch_size=64)
print(acc_test1)
acc_test2 = evaluation(shadow_pre, testloader, batch_size=4)
print(acc_test2)

tensor(0.9742)
tensor(0.9756)


In [14]:
shadow = shadow_pre
target = target_pre

#### *Create dataset for attack model*

In [15]:
# 1. create train dataset for attack model
# use shadow model and dataset: out_shadow, train_shadow
print('Creating training dataset for attack model:')
train_attack_set = get_attackdataset(shadow,shadow_train_loader,shadow_out_loader,device=device)
train_attack = attack_dataset(attack_set=train_attack_set,device=device)
print('- - - - - - - - - - - - - - - - - - ')
print('Creating test dataset for attack model:')
# 2. create test dataset for attack model
# use target model and dataset: out_target, train_target
test_attack_set = get_attackdataset(target,target_train_loader,target_out_loader,device=device)
test_attack = attack_dataset(attack_set=test_attack_set,device=device)

Creating training dataset for attack model:
(tensor([1.0000e+00, 4.1738e-08, 1.2873e-12]), tensor([0.]))
- - - - - - - - - - - - - - - - - - 
Creating test dataset for attack model:
(tensor([1.0000e+00, 1.0961e-09, 1.0551e-09]), tensor([1.]))


#### *Attack model 1* 

##### *Prepare dataset for attack model 1: LGBMClassifier*

In [16]:
train_feature = []
train_label = []
for i in train_attack_set:
  train_feature.append(i[0].cpu().numpy())
  train_label.append(int(i[1]))

test_feature = []
test_label = []
for i in test_attack_set:
  test_feature.append(i[0].cpu().numpy())
  test_label.append(int(i[1]))

##### *Attack model 1: LGBMClassifier and output*

In [20]:
attack_model = lgb.LGBMClassifier(objective='binary', reg_lambda=10, n_estimators=10000)
attack_model.fit(train_feature, train_label)
predict = attack_model.predict(test_feature)
precision_general, recall_general, _, _ = precision_recall_fscore_support(y_pred=predict,y_true=test_label,average="binary")
print('End fitting and predicting')
print('- - - - - - - - - - - - - - - ')

print('result')
print('- - - - - - - - - - - - - - - ')
print('precision: {:.5f}'.format(precision_general))
print('recall: {:.5f}'.format(recall_general))

End fitting and predicting
- - - - - - - - - - - - - - - 
result
- - - - - - - - - - - - - - - 
precision: 0.49924
recall: 0.50193


#### *Attack model 2*

Result is not good. Need more tuning or trick. Guess one possible way is to do some transforms on feature vectors such as log. 

##### *Define a MLP class for attack model 2*

In [21]:
class MLP(nn.Module):
        # 3 features, thus input size=3
        # Yes or no, and use sigmoid thus output size=1
        # If use softmax, output size = 2
        # Loss: cross entropy has contained softmax 
    def __init__(self, num_of_features=3):
        super().__init__()
        self.fc1 = nn.Linear(num_of_features, 64)
        self.fc2 = nn.Linear(64, 2)
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Hidden layer
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x,dim=-1)
        return x

##### *Functions for training and evaluating (attack model 2)*

In [22]:
def attack_train(model, dataloader, criterion, opt):
    running_loss = 0
    # switch to model:train
    # no difference here since no dropout and BN
    model.train()
    count = 0
    for i, data in tqdm(enumerate(dataloader)):
        opt.zero_grad()
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        predict = model(imgs)
        #print(predict.dtype)
        #print(labels.dtype)
        labels = torch.flatten(labels.long())
        #print(predict.shape,labels.shape)
        #print(predict)
        #print(labels)
        loss = criterion(predict, labels)
        loss.backward()
        opt.step()
        count = i
        running_loss += loss
    return running_loss / count

def attack_evaluation(model, dataloader, batch_size=64):
    # switch to model:eval
    # no difference here since no dropout and BN    
    model.eval()
    # y_tensorlist is a list consists of some tensors
    y_true_tensorlist = []
    y_predict_tensorlist = []
    for step, (batch_x, batch_y) in enumerate(dataloader):
        batch_x,batch_y = batch_x.to(device),batch_y.to(device)
        batch_y = torch.flatten(batch_y)
        output = model(batch_x)
        #output = F.softmax(output,dim=-1)
        #print(output)
        # compare cols
        batch_y_predict = torch.argmax(output, dim=1)
        #print(output)        
        #zero = torch.zeros_like(output)
        #one = torch.ones_like(output)
        #batch_y_predict = torch.where(output > 0.5, one, zero)
        #print(batch_y_predict)
        #print(batch_y_predict)
        y_predict_tensorlist.append(batch_y_predict)        
        y_true_tensorlist.append(batch_y)
        
    # combine the tensors in the list into one
    y_true = torch.cat(y_true_tensorlist,0)
    y_predict = torch.cat(y_predict_tensorlist,0)

    # compute accuracy
    length = len(y_true)
    right_length = torch.sum(y_true == y_predict)
    #print(right_length/length)
    
    return right_length/length,y_true,y_predict


##### *Train attack model 2*

In [23]:
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Train attack
attack = MLP(num_of_features=3).to(device)
# settings
#criterion = nn.BCELoss()
#criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(attack.parameters(), lr=0.00001)
epoch = 40
train_attack_dataloader = DataLoader(dataset=train_attack, batch_size=64, shuffle = True)
test_attack_dataloader = DataLoader(dataset=test_attack, batch_size=64, shuffle = True)
# train
print("start training: ")
for i in range(epoch):
    loss_train = attack_train(attack, train_attack_dataloader, criterion, opt)
    print("epoch%d   loss: %.5f  "\
      % (i+1, loss_train))
torch.save(target.state_dict(), "./models/attack_MNIST.pth")

start training: 


469it [00:00, 1052.61it/s]


epoch1   loss: 0.69482  


469it [00:00, 1027.84it/s]


epoch2   loss: 0.69456  


...

epoch38   loss: 0.69431  


469it [00:00, 1053.56it/s]


epoch39   loss: 0.69430  


469it [00:00, 1056.25it/s]


epoch40   loss: 0.69430  


In [24]:
_,y_true,y_predict = attack_evaluation(attack, test_attack_dataloader, batch_size=64)

In [25]:
precision_score(y_true.cpu().numpy(), y_predict.cpu().numpy(), average='binary')

0.5039650371832376

In [26]:
recall_score(y_true.cpu().numpy(), y_predict.cpu().numpy(), average='binary')

0.9532666666666667